# Алгоритм Эрли

Алгоритм Эрли был придуман Джеем Эрли (Jay Earley) в 1968 году. Он позволяет по слову узнать, выводимо ли оно в заданной контекстно-свободной грамматике (КС-грамматике). Алгоритм строит списки разбора, которые в свою очередь состоят из списка
ситуаций. Для построения очередного списка разбора используется ранее построенные списки, поэтому алгоритм Эрли можно
считать динамическим методом.

Сильной стороной алгоритма Эрли является то, что он может использовать любую КС-грамматику 
для разбора входной цепочки, он также генерирует лес разбора, из которого можно найти все (возможно даже бесконечное количество)
деревья разбора для данной грамматики.

*Алфавитом* в данной реализации будем считать любую непустую строку из символов Unicode. Для удобства элементы алфавита будем называть токенами.
*Словом* будем считать цепочку токенов.
*Нетерминалом* будем считать те токены, которая встречаются в левой части правил.
Для удобства можно начинать нетерминал с символа '<' и оканчивать символом '>'.

Все правила будем задавать в виде словаря языка питон:
```
grammar = {
  "<нетерминал1>": [[цепочка1], [цепочка2], ...],
  "<нетерминал2>": [[цепочка3], [цепочка4], ...],
},
```
где цепочка - это последовательность терминалов и нетерминалов.
Такая схема позволит избежать разбора записи грамматики, так как она задаётся последством конструкции языка Python 3.
Первый нетерминал по умолчанию считается стартовым.
Пример грамматики для простого арифметического выражения (последовательность сложений и умножений в 4-ричной системе счисления):

In [1]:
from importlib import reload
import formlang.earley as earley

grammar = {
    '<старт>': [['<выражение>']],
    '<выражение>': [
        ['<слагаемое>', '+', '<выражение>'],
        ['<слагаемое>']
    ],
    '<слагаемое>': [
        ['<множитель>', '*', '<множитель>'],
        ['<множитель>']
    ],
    '<множитель>': [['<число>']],
    '<число>': [
        ['<цифра>', '<число>'],
        ['<цифра>']
    ],
    '<цифра>': [[str(i)] for i in range(4)],
}

## Список ситуаций

Пусть $G=\left<N,\Sigma,P,S\right>$ — контекстно-свободная грамматика и $w=w_0w_1…w_{n−1}$ — входная цепочка из $\Sigma^*$.
Объект вида $\left[A\to\alpha\bullet\beta,i\right],$ где $A\to\alpha\bullet\beta$ — правило из $P$ и $0\leq i\leq n$ —
позиция в $w,$ называется ситуацией, относящейся к цепочке $w,$ где $\bullet$ — вспомогательный символ, который не явлется терминалом или нетерминалом $(\bullet\not\in N\cup\Sigma)$.

Ситуации хранятся в множествах $D_0,\ldots,D_{n−1}$, называемых списками ситуаций. Причем наличие ситуации
$\left[A\to\alpha\bullet\beta,i\right]$ в $j$-м списке ситуаций $D_j$ равносильно тому, что
$\exists\delta\in N\cup\Sigma:((S\Rightarrow^*w_0\ldots w_{i-1}A\delta)\land A\Rightarrow^*w_i\ldots w_{j-1})$.

Ситуации хранятся в классе State. Причём символ $\bullet$ не включется в цепочку правила, а хранится в виде позиции 
dot_index. Основные свойства класса State:
* name: неперминал, представляющий правило,
* expr: цепочка терминалов и нетерминалов,
* dot_index: положение • внутри правила,
* start_column: список ситуаций (column), откуда правило стало приминяться,
* end_column: список ситуаций, где правило завершилось.

Класс Column хранит список ситуаций. Каждый список ситуаций соответствует определённому токену.
Отметим, что каждому считанному символу из входной цепоски соответствует свой список ситуаций. Таким образом
начальный список ситуаций соответствует входной цепочке, из которой не считано ни одного символа. Основные свойства класса Column:
* index: позиция ещё не обработанного токена из входной цепочки,
* token: обработанный token в позиции index или None для index = 0.

Список ситуаций позволяет добавлять новые ситуации и проверяет их на уникальность. Не уникальные состояние
свидетельствуют о левой рекурсии и проверка на уникальность позволяет свернуть её.

Вспомогательная функция is_nt проверяeт что строка является нетерминалом, то есть присутствует в левой стороне правил.

In [2]:
print(earley.is_nt('<старт>', grammar))
print(earley.is_nt('1', grammar))

True
False


Данная реализация алгоритма Эрли обрабатывает ε-порождающие нетерминалы специальным образом.
Нетерминал $A$ называется ε-порождающим, если $A\Rightarrow^*\varepsilon$. То есть из нетерминала
выводится пустая цепочка. Это означает, что в правиле отсутствуют терминальные символы, а из всех нетерминальных
символов правила выводится пустая цепочка.

Функция nullable является реализацией одного из алгоритмов поиска ε-порождающих нетерминалов.

In [3]:
grammar_with_nullables = {
    'S': [['A', 'B']],
    'A': [['a']],
    'B': [['b'], [], ['C']],
    'C': [['A'], ['B']]
}
print(earley.nullable(grammar_with_nullables))

{'C', 'B'}


Ещё один пример:
* S→ABC
* S→DS
* A→ε
* B→AC
* C→ε
* D→d

In [4]:
another_grammar_with_nullables = {
    'S': [['A', 'B', 'C'], ['D', 'S']],
    'A': [[]],   
    'B': [['A', 'C']],
    'C': [[]],
    'D': [['d']]
}
print(earley.nullable(another_grammar_with_nullables))

{'S', 'C', 'B', 'A'}


Вывод таблицы грамматического разбора:

In [5]:
reload(earley)
grammar = {
    '<старт>': [['<выражение>']],
    '<выражение>': [
        ['<слагаемое>', '+', '<выражение>'],
        ['<слагаемое>']
    ],
    '<слагаемое>': [
        ['<множитель>', '*', '<множитель>'],
        ['<множитель>']
    ],
    '<множитель>': [['<число>']],
    '<число>': [
        ['<цифра>', '<число>'],
        ['<цифра>']
    ],
    '<цифра>': [[str(i)] for i in range(4)],
}
parser = earley.EarleyParser(grammar)
parser.recognize('1+2+3')
print(parser.get_table())

[
  None: chart[0]
    <старт> -> • <выражение> (0, 0)
    <выражение> -> • <слагаемое> + <выражение> (0, 0)
    <выражение> -> • <слагаемое> (0, 0)
    <слагаемое> -> • <множитель> * <множитель> (0, 0)
    <слагаемое> -> • <множитель> (0, 0)
    <множитель> -> • <число> (0, 0)
    <число> -> • <цифра> <число> (0, 0)
    <число> -> • <цифра> (0, 0)
    <цифра> -> • 0 (0, 0)
    <цифра> -> • 1 (0, 0)
    <цифра> -> • 2 (0, 0)
    <цифра> -> • 3 (0, 0), 
  '1': chart[1]
    <цифра> -> 1 • (0, 1)
    <число> -> <цифра> • <число> (0, 1)
    <число> -> <цифра> • (0, 1)
    <число> -> • <цифра> <число> (1, 1)
    <число> -> • <цифра> (1, 1)
    <множитель> -> <число> • (0, 1)
    <цифра> -> • 0 (1, 1)
    <цифра> -> • 1 (1, 1)
    <цифра> -> • 2 (1, 1)
    <цифра> -> • 3 (1, 1)
    <слагаемое> -> <множитель> • * <множитель> (0, 1)
    <слагаемое> -> <множитель> • (0, 1)
    <выражение> -> <слагаемое> • + <выражение> (0, 1)
    <выражение> -> <слагаемое> • (0, 1)
    <старт> -> <выражение> • 

Пример неудавшегося разбора:

In [6]:
parser.recognize('1+2+9', next(iter(grammar)))

SyntaxError: at '+9' (<string>)

Генерация дкрква разбора. Реализовано две функции, первая возвращает одно из ддеревьев:

In [7]:
parser = earley.EarleyParser(grammar)
for tree in parser.one_parse_tree('1+2+0', '<старт>'):
    earley.display_tree(tree)

<старт>
└─ <выражение>
    ├─ <слагаемое>
    │   └─ <множитель>
    │       └─ <число>
    │           └─ <цифра>
    │               └─ '1'
    ├─ '+'
    └─ <выражение>
        ├─ <слагаемое>
        │   └─ <множитель>
        │       └─ <число>
        │           └─ <цифра>
        │               └─ '2'
        ├─ '+'
        └─ <выражение>
            └─ <слагаемое>
                └─ <множитель>
                    └─ <число>
                        └─ <цифра>
                            └─ '0'


Другая возвращает все возможные деревья (возможно даже бесконечное количество). Для этого грамматику нужно записать в альтернативном виде.

In [10]:
grammar2 = {
    '<старт>': [['<выражение>']],
    '<выражение>': [
        ['<выражение>', '+', '<выражение>'],
        ['<выражение>', '*', '<выражение>'],
        ['<число>']
    ],
    '<число>': [['<цифры>']],
    '<цифры>': [
        ['<цифра>', '<цифры>'],
        ['<цифра>']
    ],
    '<цифра>': [[str(i)] for i in range(4)],
}
parser = earley.EarleyParser(grammar2)
for tree in parser.all_parse_trees('1+2+0', '<старт>'):
    earley.display_tree(tree)

<старт>
└─ <выражение>
    ├─ <выражение>
    │   ├─ <выражение>
    │   │   └─ <число>
    │   │       └─ <цифры>
    │   │           └─ <цифра>
    │   │               └─ '1'
    │   ├─ '+'
    │   └─ <выражение>
    │       └─ <число>
    │           └─ <цифры>
    │               └─ <цифра>
    │                   └─ '2'
    ├─ '+'
    └─ <выражение>
        └─ <число>
            └─ <цифры>
                └─ <цифра>
                    └─ '0'
<старт>
└─ <выражение>
    ├─ <выражение>
    │   └─ <число>
    │       └─ <цифры>
    │           └─ <цифра>
    │               └─ '1'
    ├─ '+'
    └─ <выражение>
        ├─ <выражение>
        │   └─ <число>
        │       └─ <цифры>
        │           └─ <цифра>
        │               └─ '2'
        ├─ '+'
        └─ <выражение>
            └─ <число>
                └─ <цифры>
                    └─ <цифра>
                        └─ '0'
